In [31]:
#Import Modules
simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from pathlib import Path
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.preprocessing import StandardScaler
import hvplot.pandas
from sklearn.cluster import KMeans

In [32]:
pd.set_option('display.max_columns', None)

clean_df = pd.read_csv('clean_df.csv')

In [33]:
clean_df.head()

,Unnamed: 0,name,rating,genre,year,score,votes,director,writer,star,country,budget,gross,company,runtime,formatted_date,factors,success
0,0,The Shining,R,Drama,1980,8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0,06-13-1980,2.473620,False
1,1,The Blue Lagoon,R,Adventure,1980,5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853106.0,Columbia Pictures,104.0,07-02-1980,13.078468,True
2,2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375067.0,Lucasfilm,124.0,06-20-1980,29.909726,True
3,3,Airplane!,PG,Comedy,1980,7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,83453539.0,Paramount Pictures,88.0,07-02-1980,23.843868,True
4,4,Caddyshack,R,Comedy,1980,7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0,07-25-1980,6.641057,True


In [16]:
# # Splitting the released date column and extracting the date
# def extract_date(date_str):
#     date_only_str = date_str.split('(')[0].strip()  # Extracting date part
#     date_obj = pd.to_datetime(date_only_str, format="%B %d, %Y", errors='coerce')
#     return date_obj

# # Apply the function to the 'released' column
# trimmed_df['release_date'] = trimmed_df['released'].apply(extract_date)

# trimmed_df['formatted_date'] = trimmed_df['release_date'].dt.strftime("%m-%d-%Y")
# clean_df = trimmed_df.drop(columns=['release_date', 'released'])

# # Display the DataFrame with only the date extracted
# clean_df.head()

In [17]:
# # Determining the success/failure of the movie
# clean_df['factors']= (clean_df['gross'] / clean_df['budget'])
# clean_df.head()

In [18]:
# # Creating a new column and categorizing it (success or failure) based on the condition specified
# clean_df['success'] = clean_df['factors'].map(lambda x: x>3).astype(bool)
# clean_df.head()

In [34]:
# KNN Time

# Split target column from dataset
y = clean_df['success']
X = clean_df.drop(columns='success')

In [35]:
# Insert dummies for categorical values
X = pd.get_dummies(X)

In [36]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [37]:
# X_train.shape
# X_test.shape

In [38]:
# Create the StandardScaler instance
scaler = StandardScaler()
# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)
# Scale the training data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [96]:
# Instantiate the model with k = 3 neighbors
model = KNeighborsClassifier(n_neighbors=4)

In [97]:
# Train the model
model.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=4)

In [98]:
# Create predictions
y_pred = model.predict(X_test_scaled)

# Review the predictions
y_pred

array([False, False, False, ..., False, False, False])

In [99]:
# Print confusion matrix
confusion_matrix(y_pred,y_test)

array([[857, 389],
       [ 54,  46]], dtype=int64)

In [100]:
# Print classification report
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

       False       0.94      0.69      0.79      1246
        True       0.11      0.46      0.17       100

    accuracy                           0.67      1346
   macro avg       0.52      0.57      0.48      1346
weighted avg       0.88      0.67      0.75      1346



In [44]:
# Create a a list to store inertia values and the values of k
inertia = []
k = list(range(1, 11))

In [63]:
# New DF for values to test inertia
temp_df = clean_df[['year', 'score', 'votes', 'budget', 'gross', 'runtime', 'success']]

In [65]:
temp_df.head()

,year,score,votes,budget,gross,runtime,success
0,1980,8.4,927000.0,19000000.0,46998772.0,146.0,False
1,1980,5.8,65000.0,4500000.0,58853106.0,104.0,True
2,1980,8.7,1200000.0,18000000.0,538375067.0,124.0,True
3,1980,7.7,221000.0,3500000.0,83453539.0,88.0,True
4,1980,7.3,108000.0,6000000.0,39846344.0,98.0,True


In [66]:
# Create a for-loop where each value of k is evaluated using the K-means algorithm
# Fit the model using the service_ratings DataFrame
# Append the value of the computed inertia from the `inertia_` attribute of the KMeans model instance
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(temp_df)
    inertia.append(k_model.inertia_)

In [67]:
 # Define a DataFrame to hold the values for k and the corresponding inertia
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

# Review the DataFrame
df_elbow.head()

,k,inertia
0,1,1.991094e+20
1,2,7.725989e+19
2,3,4.143214e+19
3,4,2.869043e+19
4,5,1.945481e+19


In [68]:
 # Plot the DataFrame
df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)

:Curve   [k]   (inertia)